# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
S. Li  ->  S. Li  |  ['S. Li']
K. Lee  ->  K. Lee  |  ['K. Lee']


J. Li  ->  J. Li  |  ['J. Li']
J. Liu  ->  J. Liu  |  ['J. Liu']
Arxiv has 74 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.16760
extracting tarball to tmp_2407.16760...

 done.
Retrieving document from  https://arxiv.org/e-print/2407.17012


extracting tarball to tmp_2407.17012...

 done.
Retrieving document from  https://arxiv.org/e-print/2407.17019
extracting tarball to tmp_2407.17019...

 done.
  0: tmp_2407.17019/main.tex, 55 lines
  2: tmp_2407.17019/spie-proceedings-style/main_template.tex, 251 lines
Retrieving document from  https://arxiv.org/e-print/2407.17386
extracting tarball to tmp_2407.17386...

/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2407.17019/spie-proceedings-style/main_template.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


 done.
Retrieving document from  https://arxiv.org/e-print/2407.17462
extracting tarball to tmp_2407.17462... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.16760-b31b1b.svg)](https://arxiv.org/abs/2407.16760) | **Validation of Gaia DR3 orbital and acceleration solutions with hierarchical triples**  |
|| P. Nagarajan, <mark>K. El-Badry</mark> |
|*Appeared on*| *2024-07-25*|
|*Comments*| *14 pages, 7 figures, Submitted to PASP. Github repository at this https URL*|
|**Abstract**|            Using data from Gaia DR3, we construct a sample of 14,791 gravitationally bound wide pairs in which one of the components is an unresolved binary with an astrometric orbital or acceleration solution. These systems are hierarchical triples, with inner binary separations of order $1$ au, and outer separations of $100$-$100,000$ au. Leveraging the fact that the inner binary and outer tertiary should have nearly identical parallaxes, we use the sample to calibrate the parallax uncertainties for orbital and acceleration binary solutions. We find that the parallax uncertainties of orbital solutions are typically underestimated by a factor of $1.3$ at $G>14$, and by a factor of $1.7$ at $G = 8$-$14$. The true parallax uncertainties are nevertheless a factor of $\sim 10$ smaller than those of the single-star astrometric solutions for the same sources. The parallax uncertainties of acceleration solutions are underestimated by larger factors of $2$-$3$ but still represent a significant improvement compared to the sources' single-star solutions. We provide tabulated uncertainty inflation factors for astrometric binary solutions and make the catalog of hierarchical triples publicly available.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.17012-b31b1b.svg)](https://arxiv.org/abs/2407.17012) | **Effects of Dust Coagulation on Streaming Instability**  |
|| K. W. Ho, H. Li, <mark>S. Li</mark> |
|*Appeared on*| *2024-07-25*|
|*Comments*| *7 pages, 4 figures, 1 table*|
|**Abstract**|            Streaming Instability (SI) in dust has long been thought to be a promising process in triggering planetesimal formation in the protoplanetary disks (PPDs). In this study, we present the first numerical investigation that models the SI in the vertically stratified disk together with the dust coagulation process. Our simulations reveal that, even with the initial small dust sizes, because dust coagulation promotes dust size growth, SI can eventually still be triggered. As such, the dust coagulation process broadens the parameter boundaries obtained from the previous SI studies using single dust species. We describe the various stages of dust dynamics along with their size evolution, and explore the impact of different dust fragmentation velocities. Implications of these results for realistic PPDs are also discussed.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.17019-b31b1b.svg)](https://arxiv.org/abs/2407.17019) | **Commissioning the CMB polarization telescope GroundBIRD with the full set of detectors**  |
|| M. Tsujii, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2024-07-25*|
|*Comments*| *Event: SPIE Astronomical Telescopes + Instrumentation, 2024, Yokohama, Japan; paper number 13102-7, Millimeter, Submillimeter, and Far-Infrared Detectors and Instrumentation for Astronomy XII*|
|**Abstract**|            GroundBIRD is a ground-based cosmic microwave background (CMB) experiment for observing the polarization pattern imprinted on large angular scales ($\ell > 6$ ) from the Teide Observatory in Tenerife, Spain. Our primary scientific objective is a precise measurement of the optical depth $\tau$ ($\sigma(\tau) \sim 0.01$) to the reionization epoch of the Universe to cross-check systematic effects in the measurements made by previous experiments. GroundBIRD observes a wide sky area in the Northern Hemisphere ($\sim 40\%$ of the full sky) while continuously rotating the telescope at a high speed of up to 20 rotations per minute (rpm) to overcome the fluctuations of atmospheric radiation. We have adopted the NbTiN/Al hybrid microwave kinetic inductance detectors (MKIDs) as focal plane detectors. We observe two frequency bands centered at 145 GHz and 220 GHz. The 145 GHz band picks up the peak frequency of the CMB spectrum. The 220 GHz band helps accurate removal of the contamination of thermal emission from the Galactic interstellar dust. The MKID arrays (138 MKIDs for 145GHz and 23 MKIDs for 220GHz) were designed and optimized so as to minimize the contamination of the two-level-system noise and maximize the sensitivity. The MKID arrays were successfully installed in May 2023 after the performance verification tests were performed at a laboratory. GroundBIRD has been upgraded to use the full MKID arrays, and scientific observations are now underway. The telescope is automated, so that all observations are performed remotely. Initial validations, including polarization response tests and observations of Jupiter and the moon, have been completed successfully. We are now running scientific observations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.17386-b31b1b.svg)](https://arxiv.org/abs/2407.17386) | **Data-driven stellar intrinsic colors and dust reddenings for spectro-photometric data: From the blue-edge method to a machine-learning approach**  |
|| H. Zhao, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-07-25*|
|*Comments*| *23 pages, 1 table, 11 figures, 2 appendices, accepted for publication in ApJ*|
|**Abstract**|            Intrinsic colors (ICs) of stars are essential for the studies on both stellar physics and dust reddening. In this work, we developed an XGBoost model to predict the ICs with the atmospheric parameters $T_{\rm eff}$, ${\rm log}\,g$, and $\rm [M/H]$. The model was trained and tested for three colors at Gaia and 2MASS bands with 1,040,446 low-reddening sources. The atmospheric parameters were determined by the Gaia DR3 GSP-phot module and were validated by comparing with APOGEE and LAMOST. We further confirmed that the biases in GSP-phot parameters, especially for $\rm [M/H]$, do not present a significant impact on the IC prediction. The generalization error of the model estimated by the test set is 0.014 mag for $(G_{\rm BP}\,{-}\,G_{\rm RP})_0$, 0.050 mag for $(G_{\rm BP}\,{-}\,K_{\rm S})_0$, and 0.040 mag for $(J\,{-}\,K_{\rm S})_0$. The model was applied to a sample containing 5,714,528 reddened stars with stellar parameters from Andrae et al. (2023) to calculate ICs and reddenings. The high consistency in the comparison of $E(J\,{-}\,K_{\rm S})$ between our results and literature values further validates the accuracy of the XGBoost model. The variation of $E(G_{\rm BP}\,{-}\,K_{\rm S})/E(G_{\rm BP}\,{-}\,G_{\rm RP})$, a representation of the extinction law, with Galactic longitude is found on large scales. This work preliminarily presents the feasibility and the accuracy of the machine-learning approach for IC and dust reddening calculation, whose products could be widely applied to spectro-photometric data. The data sets and trained model can be accessed via \url{this https URL}. The models for more bands will be completed in the following works.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.17462-b31b1b.svg)](https://arxiv.org/abs/2407.17462) | **The HalfDome Multi-Survey Cosmological Simulations: N-body Simulations**  |
|| A. E. Bayer, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-07-25*|
|*Comments*| *10 pages, 7 figures, 2 tables. Data released at this https URL*|
|**Abstract**|            Upcoming cosmological surveys have the potential to reach groundbreaking discoveries on multiple fronts, including the neutrino mass, dark energy, and inflation. Most of the key science goals require the joint analysis of datasets from multiple surveys to break parameter degeneracies and calibrate systematics. To realize such analyses, a large set of mock simulations that realistically model correlated observables is required. In this paper we present the N-body component of the HalfDome cosmological simulations, designed for the joint analysis of Stage-IV cosmological surveys, such as Rubin LSST, Euclid, SPHEREx, Roman, DESI, PFS, Simons Observatory, CMB-S4, and LiteBIRD. Our 300TB initial data release includes full-sky lightcones and halo catalogs between $z$=0--4 for 11 fixed cosmology realizations, as well as an additional run with local primordial non-Gaussianity ($f_{\rm NL}$=20). The simulations evolve $6144^3$ particles in a 3.75$\,h^{-1} {\rm Gpc}$ box, reaching a minimum halo mass of $\sim 6 \times 10^{12}\,h^{-1} M_\odot$ and maximum scale of $k \sim 1\,h{\rm Mpc}^{-1}$. Instructions to access the data, and plans for future data releases, can be found at this https URL.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

146  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
